In [1]:
import json
from time import sleep
import requests
import lxml
from bs4 import BeautifulSoup
from selenium import webdriver
import sqlite3 as lite
import pandas as pd
from pymongo import MongoClient

# Vacation Homework

## Q1. Python Basic, Web Scraping, RDBMS, SQL

### Q1-1

`Ancient Egyptian multiplication`(one of two multiplication methods used by scribes, was a systematic method for multiplying two numbers that does not require the multiplication table, only the ability to multiply and divide by 2, and to add.) 을 Python 으로 구현하세요.

example output)

```
enter two numbers: 24 15
  24      15 struck
  12      30 struck
   6      60 struck
   3     120 keep
   1     240 keep
The result is 360
```

In [10]:
# Q1-1
user_input = input("Enter Two Numbers: ")
[a, b] = [int(x) for x in user_input.split()]
kept_b = []

while a > 0:
    if a%2 == 1:
        status = "keep"
        kept_b.append(b)
    else:
        status = "struck"
        
    print("{a}{tab}{b}{tab}{status}".format(a=a, tab='\t', b=b, status=status))
    a //= 2
    b *= 2

result = sum([i for i in kept_b])
print("The result is {}".format(result))

Enter Two Numbers: 24 15
24	15	struck
12	30	struck
6	60	struck
3	120	keep
1	240	keep
The result is 360


### Q1-2
Twitter의 #asiancup 에 대한 트윗 검색결과(https://twitter.com/search?q=%23asiancup)를 sqlite 를 활용한 database에 적재하세요. (10회 스크롤 했을 때의 트윗 본문 전체(이미지 제외))

Requirements: selenium, sqlite3

Caution : https://twitter.com/robots.txt 의 지침을 따르세요.

```
# Every bot that might possibly read and respect this file.
User-agent: *
Allow: /*?lang=
Allow: /hashtag/*?src=
Allow: /search?q=%23
Disallow: /search/realtime
Disallow: /search/users
Disallow: /search/*/grid

Disallow: /*?
Disallow: /*/followers
Disallow: /*/following

Disallow: /account/not_my_account
Disallow: /account/deactivated
Disallow: /settings/deactivated

Disallow: /oauth
Disallow: /1/oauth

Disallow: /i/streams
Disallow: /i/hello

# Wait 1 second between successive requests. See ONBOARD-2698 for details.
Crawl-delay: 1
```

#### Practice

In [28]:
ch_driver = webdriver.Chrome('C:/Users/beave/AppData/Roaming/Microsoft/Windows/Start Menu/Programs/Python 3.7/chromedriver.exe')
url = 'https://twitter.com/search?q=%23asiancup'
ch_driver.get(url)

In [29]:
articles = ch_driver.find_elements_by_class_name('content')

# article-id and time
time = articles[0].find_element_by_class_name('time')
time_a = time.find_element_by_tag_name('a')
post_time = time_a.get_attribute('title') #time
post_id = time_a.get_attribute('data-conversation-id') #article-id

# body
body_div = articles[0].find_element_by_class_name("js-tweet-text-container")
post_body = body_div.find_element_by_tag_name("p").text

print(post_time, post_id, post_body, sep='\n')

오후 4:03 - 2019년 1월 23일
1088225691332534272
1일 베트남인. 쳐부수자 일제 원숭이!!!
Vietnam vo dich! 
#AsianCup #Vietnam


#### Web Crawling

In [31]:
ch_driver = webdriver.Chrome('C:/Users/beave/AppData/Roaming/Microsoft/Windows/Start Menu/Programs/Python 3.7/chromedriver.exe')
url = 'https://twitter.com/search?q=%23asiancup'
ch_driver.get(url)

result = []

for _ in range(10):
    articles = ch_driver.find_elements_by_class_name('content')
    for item in articles:
        time = item.find_element_by_class_name('time')
        time_a = time.find_element_by_tag_name('a')
        
        post_time = time_a.get_attribute('title') #time
        post_id = time_a.get_attribute('data-conversation-id') #article-id
        
        body_div = item.find_element_by_class_name("js-tweet-text-container")
        post_body = body_div.find_element_by_tag_name("p").text
    
        result.append((post_time, post_id, post_body))
    
    ch_driver.execute_script("window.scrollTo(0, 1080)") # scroll down
    sleep(2) 

In [33]:
# for check
result[:10]

[('오후 4:03 - 2019년 1월 23일',
  '1088225691332534272',
  '1일 베트남인. 쳐부수자 일제 원숭이!!!\nVietnam vo dich! \n#AsianCup #Vietnam'),
 ('오후 11:04 - 2019년 1월 24일',
  '1088694033025650688',
  '오늘은 킹덤 DDay 17:00 #Netflix\n밤엔 축구 DDay 22:00 #Asiancup'),
 ('오전 6:54 - 2019년 1월 25일',
  '1088812295445250052',
  '아쉽다... 한일로의 결승전을 기대했는데...... #asiancup #대한민국'),
 ('오후 7:21 - 2019년 1월 25일',
  '1089000442112012291',
  '"미안한 생각이 많이 드는 것 같습니다"\n#손흥민 #asiancup pic.twitter.com/qmzoCqVbC8'),
 ('오전 7:23 - 2019년 1월 25일',
  '1088819559971385344',
  '아시안컵 탈락이다..................\n#AsianCup'),
 ('오전 9:21 - 2019년 1월 25일',
  '1088849413714276352',
  'ㅈ..진정해요..!\n#김영권 AsianCup2019 #asiancup '),
 ('오후 11:40 - 2019년 1월 22일',
  '1087978282635718656',
  '아시아쿼터 찾는 수원, 아시안컵 공격수 관찰 중 (출처 : 스포티비뉴스 | 네이버 스포츠) \nK league Suwon Samsung, Asian Cup striker watching\nhttp://naver.me/54M4YBpd  \n#K리그 #Kleague #수원삼성 #suwonsamsung #아시안컵 #asiancup'),
 ('오후 7:53 - 2019년 1월 25일',
  '1089008350170206208',
  '"선수들 그리고 코칭스태프들한테 폐를 끼친 것 같다 미안하다는 생각

#### Insert Data Using SQLite

1. Create twitter_posts.db on sqlite3 and move db to your workspace<br/>
`.open twitter.db`

2. Insert the result into db with Jupyter Notebook

In [34]:
db = lite.connect('twitter.db')

In [35]:
pd.options.display.max_colwidth = 300

In [37]:
query_create_table = """
CREATE TABLE IF NOT EXISTS
Asiancup 
("Datetime" TEXT,
"ArticleID" TEXT,
"ArticleBody" TEXT);
"""

query_insert_data = """
INSERT INTO Asiancup VALUES(?,?,?);
"""

with db:
    cur = db.cursor()
    cur.execute(query_create_table)
    for row in result:
        cur.execute(query_insert_data,row)
        
pd.read_sql("SELECT * FROM Asiancup;", db).head()

,Datetime,ArticleID,ArticleBody
0,오후 4:03 - 2019년 1월 23일,1088225691332534272,1일 베트남인. 쳐부수자 일제 원숭이!!!\nVietnam vo dich! \n#AsianCup #Vietnam
1,오후 11:04 - 2019년 1월 24일,1088694033025650688,오늘은 킹덤 DDay 17:00 #Netflix\n밤엔 축구 DDay 22:00 #Asiancup
2,오전 6:54 - 2019년 1월 25일,1088812295445250052,아쉽다... 한일로의 결승전을 기대했는데...... #asiancup #대한민국
3,오후 7:21 - 2019년 1월 25일,1089000442112012291,"""미안한 생각이 많이 드는 것 같습니다""\n#손흥민 #asiancup pic.twitter.com/qmzoCqVbC8"
4,오전 7:23 - 2019년 1월 25일,1088819559971385344,아시안컵 탈락이다..................\n#AsianCup


## Q2. SQL, RDBMS

`products.db` 에 대하여 다음 문제를 해결하세요.

In [38]:
products_db = lite.connect('products.db')

### Q2-1

`products.db`에 존재하는 모든 `table`과 `column`을 조회하세요.

In [40]:
# Q2-1
query = """
SELECT tbl_name, sql FROM sqlite_master;
"""

pd.read_sql(query, products_db)

,tbl_name,sql
0,Categories,"CREATE TABLE Categories(\n ""CategoryID"" TEXT,\n ""CategoryName"" TEXT,\n ""Description"" TEXT\n)"
1,Customers,"CREATE TABLE Customers(\n ""CustomerID"" TEXT,\n ""CustomerName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""City"" TEXT,\n ""PostalCode"" TEXT,\n ""Country"" TEXT\n)"
2,Employees,"CREATE TABLE Employees(\n ""EmployeeID"" TEXT,\n ""LastName"" TEXT,\n ""FirstName"" TEXT,\n ""BirthDate"" TEXT,\n ""Photo"" TEXT,\n ""Notes"" TEXT\n)"
3,Shippers,"CREATE TABLE Shippers(\n ""ShipperID"" TEXT,\n ""ShipperName"" TEXT,\n ""Phone"" TEXT\n)"
4,Suppliers,"CREATE TABLE Suppliers(\n ""SupplierID"" TEXT,\n ""SupplierName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""City"" TEXT,\n ""PostalCode"" TEXT,\n ""Country"" TEXT,\n ""Phone"" TEXT\n)"
5,Orders,"CREATE TABLE Orders(\nOrderID integer,\nCustomerID integer,\nEmployeeID integer,\nOrderDate date,\nShipperID integer)"
6,OrderDetails,"CREATE TABLE OrderDetails(\nOrderDetailID integer,\nOrderID integer,\nProductID integer,\nQuantity integer)"
7,Products,"CREATE TABLE Products(\nProductID integer,\nProductName text,\nSupplierID integer,\nCategoryID integer,\nUnit text,\nPrice integer\n)"


### Q2-2

`Customer`의 `Country`가 "Germany" 이거나 `City`가 "London" 인 모든 주문의 주문양(`Quantity`)의 합을 조회하세요.

In [72]:
# Q 2-2
query = """
SELECT sum(Quantity) "Total Quantity"
FROM OrderDetails
WHERE
    OrderID IN(
    SELECT OrderID
    FROM Orders
    WHERE 
        CustomerID IN(
        SELECT CustomerID
        FROM Customers
        WHERE
            Country == "Germany"
            OR
            City == "London"
        )
    )
;    
"""

pd.read_sql(query, products_db)

,Total Quantity
0,2633


### Q2-3

월별 총 주문 횟수와 총 주문 갯수, 평균 주문 금액(소수점 2째 자리에서 반올림)을 조회하세요.
(총 주문 횟수를 기준으로 내림차순 정렬)

In [91]:
query = """
SELECT 
    count(D.OrderID) "Count",
    substr(O.OrderDate, 1, 7) "Month",
    sum(D.Quantity) "Total Quantity",
    round(sum(D.Quantity * P.Price) / count(D.OrderID), 2) "Average"
    
FROM
    OrderDetails D
JOIN
    Orders O
    ON
    D.OrderID == O.OrderID
JOIN
    Products P
    ON
    D.ProductID == P.ProductID
GROUP BY
    "Month"
ORDER BY
    "Count" DESC
;
"""

pd.read_sql(query, products_db)

,Count,Month,Total Quantity,Average
0,85,1997-01,2401,981.18
1,81,1996-12,2200,786.68
2,73,1996-10,1738,705.87
3,69,1996-08,1322,482.40
4,66,1996-11,1735,941.88
5,59,1996-07,1462,640.34
6,57,1996-09,1124,606.41
7,28,1997-02,761,713.53
